In [1]:
import numpy as np
from ondelette import *
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_probability as tfp

np.set_printoptions(floatmode='unique')
dtype = tf.float32
rng = np.random.default_rng()

2025-03-24 18:56:52.738703: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 18:56:52.742419: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 18:56:52.755227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 18:56:52.780623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 18:56:52.780672: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 18:56:52.796682: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
song = 'MusicDelta_Rock'
audio = tfio.audio.AudioIOTensor(f'data/MedleyDB_V1/{song}_MIX.mp3', dtype=tf.float32)
rate = tf.cast(audio.rate, dtype=tf.int64)
raw_samples = audio.to_tensor()
samples = tf.transpose(raw_samples)
print(f'samples: {samples.shape[1]}')


2025-03-24 18:56:57.441456: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA


samples: 577320


In [3]:
for name, make in {'sym6': wavelets.sym6}.items():
    wavelet = make(dtype)
    filters = wavelets.dwt_filters(wavelet)
    inverse_filters = wavelets.idwt_filters(wavelet)

    levels = wavelets.wavedec_valid(filters, 10, samples)
    print(f'Levels: {[tuple(l.shape) for l in levels]}')
    frames = framing.levels_to_frames(levels)
    print(f'Frames: {[tuple(l.shape) for l in frames]}')
    frame_level_rms = analysis.rms_by_frame_and_level(frames)
    print(f'Frame/Level RMS: {tuple(frame_level_rms.shape)}')
    distance_matrix = analysis.distance_matrix(frame_level_rms)
    print(f'Euclidean distance matrix: {tuple(distance_matrix.shape)}')
    print(distance_matrix.numpy())



Levels: [(2, 1117), (2, 1117), (2, 2234), (2, 4468), (2, 8936), (2, 17872), (2, 35744), (2, 71488), (2, 142976), (2, 285952)]
Frames: [(1117, 2, 1), (1117, 2, 1), (1117, 2, 2), (1117, 2, 4), (1117, 2, 8), (1117, 2, 16), (1117, 2, 32), (1117, 2, 64), (1117, 2, 128), (1117, 2, 256)]
Frame/Level RMS: (1117, 10)
Euclidean distance matrix: (1117, 1117)
[[0.0000000e+00 1.3327756e+00 1.9560937e+00 ... 3.5984211e+00
  3.5827613e+00 3.6280165e+00]
 [1.3327756e+00 0.0000000e+00 1.1810921e+00 ... 2.8886342e+00
  2.8470514e+00 3.0740402e+00]
 [1.9560939e+00 1.1810925e+00 0.0000000e+00 ... 1.9685655e+00
  1.9916105e+00 2.1802001e+00]
 ...
 [3.5984211e+00 2.8886342e+00 1.9685655e+00 ... 0.0000000e+00
  2.0906422e-01 5.4258507e-01]
 [3.5827613e+00 2.8470514e+00 1.9916105e+00 ... 2.0906422e-01
  0.0000000e+00 5.8886617e-01]
 [3.6280165e+00 3.0740402e+00 2.1802001e+00 ... 5.4258484e-01
  5.8886635e-01 9.7656250e-04]]
